##PART 1

1.	Understanding the pattern



2.	Compute six-month LTV

*     LTV 1 email per week = 19.996
*     LTV 2 emails per week = 27.819
*     LTV 3 emails per week = 26.431
*     LTV 4 emails per week = 25.115

    Please refer to the spreadsheet IKEA-LTV for detailed calculation.

3.	Single email-frequency policy

    According to our LTV calculations, it indicates that the email frequency of 2 emails per week results in a highest LTV in the experiment, any further increase in frequency to three or four emails per week leads to a decline in LTV. Therefore, an email frequency of two per week is optimal.


##PART 2

1. NPTB

In [ ]:
import pandas as pd
df = pd.read_csv('IKEA.csv')
df

,customer_id,buy,order_size,message,age,female,income,home_improvement_index,space_optimization_index,number_of_Outdoor,number_of_HomeDecor,number_of_Bedroom,number_of_Storage,number_of_LivingRoom,number_of_Lighting,number_of_KitchenDining,training
0,6083030,0,0,Bedroom,>= 60,0,40000,24,0.4,0,0,0,1,0,0,1,1
1,7997633,1,108,Lighting,45 to 59,0,75000,50,1.0,0,0,2,3,3,0,1,1
2,1668615,0,0,LivingRoom,< 30,1,45000,42,0.4,0,0,1,2,2,4,2,1
3,1877853,1,442,Bedroom,45 to 59,0,75000,37,1.4,0,0,2,3,4,5,0,1
4,8425092,1,133,Outdoor,45 to 59,1,65000,38,1.4,0,0,1,2,3,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,9261480,0,0,LivingRoom,< 30,1,65000,34,1.0,0,0,1,0,3,4,6,0
249996,6553574,1,153,Bedroom,< 30,1,100000,77,1.0,0,0,4,8,10,9,9,0
249997,2591431,0,0,Lighting,>= 60,0,50000,24,1.1,0,0,0,1,0,1,0,0
249998,4694239,1,227,Lighting,30 to 44,1,70000,51,1.1,0,0,2,2,5,1,4,0


Compute the predicted probability of purchase for each of the seven departments by running the weighted logit model

In [ ]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Exclude order_size mentioned in Tip 3
formula = 'buy ~ age + female + income +	home_improvement_index + space_optimization_index + number_of_Outdoor + number_of_HomeDecor	+ number_of_Bedroom +	number_of_Storage +	number_of_LivingRoom +	number_of_Lighting + number_of_KitchenDining'

# True population positive rate (O) and oversampled rate (R)
O = 0.0084   # 0.84% in the population
R = 0.50    # 50/50 in our data


## split training data and validation data
train_df = df[df['training'] == 1].copy()
val_df = df[df['training'] == 0].copy()

# ------ Bedroom
subset_Bedroom = train_df[train_df["message"]=="Bedroom"]
# Class weights per row
w_bed = np.where(subset_Bedroom ["buy"] == 1, O/R, (1-O)/(1-R))
# run weighted logit
model_Bedroom = smf.glm(formula, data=subset_Bedroom,family=sm.families.Binomial(),freq_weights=w_bed).fit()
val_df["p_Bedroom"] = model_Bedroom.predict(val_df)

# ------ Outdoor
subset_Outdoor = train_df[train_df["message"]=="Outdoor"]
w_outdoor = np.where(subset_Outdoor["buy"] == 1, O/R, (1-O)/(1-R))
model_Outdoor = smf.glm(formula, data=subset_Outdoor,family=sm.families.Binomial(),freq_weights=w_outdoor).fit()
val_df["p_Outdoor"] = model_Outdoor.predict(val_df)

# ------ HomeDecor
subset_Home = train_df[train_df["message"]=="HomeDecor"]
w_home = np.where(subset_Home["buy"] == 1, O/R, (1-O)/(1-R))
model_Home = smf.glm(formula, data=subset_Home,family=sm.families.Binomial(),freq_weights=w_home).fit()
val_df["p_HomeDecor"] = model_Home.predict(val_df)

# ------ Storage
subset_Storage = train_df[train_df["message"]=="Storage"]
w_storage = np.where(subset_Storage["buy"] == 1, O/R, (1-O)/(1-R))
model_Storage = smf.glm(formula, data=subset_Storage,family=sm.families.Binomial(),freq_weights=w_storage).fit()
val_df["p_Storage"] = model_Storage.predict(val_df)

# ------ Living Room
subset_Living = train_df[train_df["message"]=="LivingRoom"]
w_living = np.where(subset_Living["buy"] == 1, O/R, (1-O)/(1-R))
model_Living = smf.glm(formula, data=subset_Living,family=sm.families.Binomial(),freq_weights=w_living).fit()
val_df["p_LivingRoom"] = model_Living.predict(val_df)

# ------ Lighting
subset_Lighting = train_df[train_df["message"]=="Lighting"]
w_lighting = np.where(subset_Lighting["buy"] == 1, O/R, (1-O)/(1-R))
model_Lighting = smf.glm(formula, data=subset_Lighting,family=sm.families.Binomial(),freq_weights=w_lighting).fit()
val_df["p_Lighting"] = model_Lighting.predict(val_df)

# ------ Kitchen
subset_Kitchen = train_df[train_df["message"]=="KitchenDining"]
w_kitchen = np.where(subset_Kitchen["buy"] == 1, O/R, (1-O)/(1-R))
model_Kitchen = smf.glm(formula, data=subset_Kitchen,family=sm.families.Binomial(),freq_weights=w_kitchen).fit()
val_df["p_KitchenDining"] = model_Kitchen.predict(val_df)



Compare the predicted probabilities for each customer in order to find out the right department to be assigned (which has the maximum probability of purchase)

In [ ]:
# 1. Take row-wise max
val_df["p_max"] = val_df[["p_Outdoor", "p_Bedroom", "p_HomeDecor","p_Storage","p_LivingRoom","p_Lighting","p_KitchenDining"]].max(axis=1)

# 2. Get which column had the max → right department to send message
val_df["predicted_message"] = val_df[["p_Outdoor", "p_Bedroom", "p_HomeDecor","p_Storage","p_LivingRoom","p_Lighting","p_KitchenDining"]].idxmax(axis=1)

# 3. Clean up column names so they read nicely
val_df["predicted_message"] = val_df["predicted_message"].str.replace("p_", "")


2. The distribution by department reveals that LivingRoom as the primary area of predicted message (best department), capturing 39.2% of all customers.  This analysis indicates that nearly 40% of our customer base is most likely to make purchases in the LivingRoom category, making it the most promising department for personalized recommendations.



In [ ]:
percentage_distribution = val_df['predicted_message'].value_counts(normalize=True) * 100
for department, percentage in percentage_distribution.items():
    print(f"{percentage:.1f}% {department}")

39.2% LivingRoom
37.2% KitchenDining
9.7% Bedroom
6.8% Lighting
4.5% HomeDecor
2.2% Outdoor
0.4% Storage


3. Profit Analysis